#### Ecommerce Test Site to Learn Web Scraping
- ScrapingCourse.com
- Src: https://www.scrapingcourse.com/ecommerce/

In [27]:
import csv
import requests
from bs4 import BeautifulSoup
import json

In [6]:
# find all urls products detail in page
urls = []
for i in range(1, 13):
    response = requests.get(f"https://www.scrapingcourse.com/ecommerce/page/{i}/")

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.select("a.woocommerce-loop-product__link")
        # print("Page ", i, " : ", len(links))

        for link in links:
            urls.append(link["href"])
            # print(link["href"])

In [32]:
# find product detail
products = []

for id,url in enumerate(urls):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content,"html.parser")

        # create product object
        product = {}
        product["id"] = id + 1
        product["url"] = url
        product["title"] = soup.select_one(".product_title").text.replace('"', '\\"')
        product["price"] = soup.select_one(".entry-summary .woocommerce-Price-amount").text
        product["image"] = soup.select_one(".wp-post-image")["src"]

        # check size text exist
        if soup.select_one(".woocommerce-product-attributes-item--attribute_size .woocommerce-product-attributes-item__value"):
            product["size"] = soup.select_one(".woocommerce-product-attributes-item--attribute_size .woocommerce-product-attributes-item__value").text
        else:
            product["size"] = "No size"

        # check color text exist
        if soup.select_one(".woocommerce-product-attributes-item--attribute_color .woocommerce-product-attributes-item__value"):
            product["size"] = product["color"] = soup.select_one(".woocommerce-product-attributes-item--attribute_color .woocommerce-product-attributes-item__value").text
        else:
            product["color"] = "No color"

        product["description"] = soup.select_one(".woocommerce-Tabs-panel--description p").text
        # print(product)
        products.append(product)

In [44]:
# field names
fields = ['id', 'url', 'title', 'price','image','size','color','description']

# initialize the CSV output file
with open("products.csv", "w") as csv_file:
    # creating a csv dict writer object
    writer = csv.DictWriter(csv_file,fieldnames=fields)

    # writing headers (field names)
    writer.writeheader()

    # writing data rows
    writer.writerows(products)

In [42]:
with open("data/products.json","w") as f:
    json.dump(products,f,indent=4)